<a href="https://colab.research.google.com/github/ElaYJ/Study_Deep_Learning/blob/main/Lecture/54_GPT_HF_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenizer

- 모델에 적용된 tokenizer를 사용해야 성능이 보장된다.

- 새로운 데이터셋을 해당 모델에 맞게 subword를 생성하는 것이 tokenization이다.

- tokenizer를 통해 모델이 학습할 수 있게 subword를 생성해야 한다.

</br>

### - 예제. code_net

- https://wikidocs.net/166821

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

# FutureWarning: The repository for code_search_net contains custom code
# which must be executed to correctly load the dataset.
# You can inspect the repository content at https://hf.co/datasets/code_search_net
# You can avoid this message in future by passing the argument `trust_remote_code=True`.
# Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
# 41만 개, 오래걸림
raw_datasets = load_dataset("code_search_net", "python", trust_remote_code=True)
raw_datasets

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 412178
    })
    test: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 22176
    })
    validation: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 23107
    })
})

In [3]:
raw_datasets['train']

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [4]:
# 자연여 데이터라고 할 수 없을 정도의 데이터셋이므로
# 이것을 어떻게 subword로 tokenization할지 고민해야 한다.

print(raw_datasets["train"][12]['whole_func_string'])

def _get_SRF_tau(self, imt_per):
        """
        Table 6 and equation 19 of 2013 report.
        """
        if imt_per < 1:
            srf = 0.87
        elif 1 <= imt_per < 5:
            srf = self._interp_function(0.58, 0.87, 5, 1, imt_per)
        elif 5 <= imt_per <= 10:
            srf = 0.58
        else:
            srf = 1

        return srf


- BERT 모델

    - BERT 모델을 가져와 위의 코드를 이해하는 모델링을 만들고자 한다면

        Fine Tuning을 할 때 tokenizer도 함께 학습시켜야 한다.
</br>

- GPT 모델

    - GPT model인 경우,

    - tokenizer를 학습한다는 것의 의미는...

     byte encoding --> subword를 보고, 데이터를 보면서 update --> 데이터를 보면서 iteration이 필요 (한꺼번에 가져올 수 X)

In [8]:
# 1000개씩 corpus(집대성, 언어 자료)
# 데이터를 iterator로 만드는 과정이다.

training_corpus = (
    raw_datasets['train'][i : i+1000]['whole_func_string'] for i in range(0,len(raw_datasets['train']),1000)
)

In [12]:
raw_datasets['train'][:3]['whole_func_string']

['def export_ruptures_csv(ekey, dstore):\n    """\n    :param ekey: export key, i.e. a pair (datastore key, fmt)\n    :param dstore: datastore object\n    """\n    oq = dstore[\'oqparam\']\n    if \'scenario\' in oq.calculation_mode:\n        return []\n    dest = dstore.export_path(\'ruptures.csv\')\n    header = (\'rupid multiplicity mag centroid_lon centroid_lat \'\n              \'centroid_depth trt strike dip rake boundary\').split()\n    rows = []\n    for rgetter in gen_rupture_getters(dstore):\n        rups = rgetter.get_ruptures()\n        rup_data = calc.RuptureData(rgetter.trt, rgetter.rlzs_by_gsim)\n        for r in rup_data.to_array(rups):\n            rows.append(\n                (r[\'rup_id\'], r[\'multiplicity\'], r[\'mag\'],\n                 r[\'lon\'], r[\'lat\'], r[\'depth\'],\n                 rgetter.trt, r[\'strike\'], r[\'dip\'], r[\'rake\'],\n                 r[\'boundary\']))\n    rows.sort()  # by rupture serial\n    comment = \'investigation_time=%s, ses_pe

In [13]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
example = '''def add_numbers(a, b):
    """Add the two numbers 'a' and 'b'."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 "Ġ'",
 'a',
 "'",
 'Ġand',
 "Ġ'",
 'b',
 '\'."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [16]:
len(tokens)

35

In [17]:
# tokenizer 학습 : train_new_from_iterator()

tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000) #--> vocab_size vocabulary 최대 크기

In [18]:
tokens = tokenizer.tokenize(example)
tokens, len(tokens)

(['def',
  'Ġadd',
  '_',
  'numbers',
  '(',
  'a',
  ',',
  'Ġb',
  '):',
  'ĊĠĠĠ',
  'Ġ"""',
  'Add',
  'Ġthe',
  'Ġtwo',
  'Ġnumbers',
  "Ġ'",
  'a',
  "'",
  'Ġand',
  "Ġ'",
  'b',
  '\'."""',
  'ĊĠĠĠ',
  'Ġreturn',
  'Ġa',
  'Ġ+',
  'Ġb'],
 27)

In [19]:
example = raw_datasets["train"][12]['whole_func_string']

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġ_',
 'get',
 '_',
 'SR',
 'F',
 '_',
 't',
 'au',
 '(',
 'self',
 ',',
 'Ġim',
 't',
 '_',
 'per',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'ĠTable',
 'Ġ6',
 'Ġand',
 'Ġequation',
 'Ġ19',
 'Ġof',
 'Ġ2013',
 'Ġreport',
 '.',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġif',
 'Ġim',
 't',
 '_',
 'per',
 'Ġ<',
 'Ġ1',
 ':',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġsrf',
 'Ġ=',
 'Ġ0',
 '.',
 '87',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġel',
 'if',
 'Ġ1',
 'Ġ<=',
 'Ġim',
 't',
 '_',
 'per',
 'Ġ<',
 'Ġ5',
 ':',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġsrf',
 'Ġ=',
 'Ġself',
 '._',
 'inter',
 'p',
 '_',
 'function',
 '(',
 '0',
 '.',
 '58',
 ',',
 'Ġ0',
 '.',
 '87',
 ',',
 'Ġ5',
 ',',
 'Ġ1',
 ',',
 'Ġim',
 't',
 '_',
 'per',
 ')',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',

In [20]:
len(tokens)

213

In [21]:
tokens = tokenizer.tokenize(example)
tokens

['def',
 'Ġ_',
 'get',
 '_',
 'S',
 'RF',
 '_',
 'tau',
 '(',
 'self',
 ',',
 'Ġimt',
 '_',
 'per',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġ"""',
 'ĊĠĠĠĠĠĠĠ',
 'ĠTable',
 'Ġ6',
 'Ġand',
 'Ġequation',
 'Ġ19',
 'Ġof',
 'Ġ2013',
 'Ġreport',
 '.',
 'ĊĠĠĠĠĠĠĠ',
 'Ġ"""',
 'ĊĠĠĠĠĠĠĠ',
 'Ġif',
 'Ġimt',
 '_',
 'per',
 'Ġ<',
 'Ġ1',
 ':',
 'ĊĠĠĠĠĠĠĠĠĠĠĠ',
 'Ġsrf',
 'Ġ=',
 'Ġ0',
 '.',
 '87',
 'ĊĠĠĠĠĠĠĠ',
 'Ġelif',
 'Ġ1',
 'Ġ<=',
 'Ġimt',
 '_',
 'per',
 'Ġ<',
 'Ġ5',
 ':',
 'ĊĠĠĠĠĠĠĠĠĠĠĠ',
 'Ġsrf',
 'Ġ=',
 'Ġself',
 '._',
 'interp',
 '_',
 'function',
 '(',
 '0',
 '.',
 '58',
 ',',
 'Ġ0',
 '.',
 '87',
 ',',
 'Ġ5',
 ',',
 'Ġ1',
 ',',
 'Ġimt',
 '_',
 'per',
 ')',
 'ĊĠĠĠĠĠĠĠ',
 'Ġelif',
 'Ġ5',
 'Ġ<=',
 'Ġimt',
 '_',
 'per',
 'Ġ<=',
 'Ġ10',
 ':',
 'ĊĠĠĠĠĠĠĠĠĠĠĠ',
 'Ġsrf',
 'Ġ=',
 'Ġ0',
 '.',
 '58',
 'ĊĠĠĠĠĠĠĠ',
 'Ġelse',
 ':',
 'ĊĠĠĠĠĠĠĠĠĠĠĠ',
 'Ġsrf',
 'Ġ=',
 'Ġ1',
 'ĊĊĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġsrf']

In [22]:
len(tokens)

104

<br></br>

## 예제.

- https://wikidocs.net/166824

- 전체적인 tokenize 방식

    <img src="https://github.com/ElaYJ/supplement/assets/153154981/839b5a3f-a520-406c-9f89-0be4199321d8" width="57%">

#### - normalization

- 정규화 단계에는 불필요한 공백 제거, 소문자 변환(lowercasing) 및 악센트 제거 등과 같은 몇가지 일반적인 정제 작업이 포함된다.

- 유니코드 정규화(Unicode normalization) 작업과 거의 동일한 작업이 이 과정에서 수행

In [23]:
from transformers import AutoTokenizer

# BERT 기반 tokenizer 가져오기
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(type(tokenizer.backend_tokenizer))
#--> Tokenizers 라이브러리에는 하부 토크나이저에 대한 액세스를 제공하는 backend_tokenizer라는 속성을 가지고 있다.

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

<class 'tokenizers.Tokenizer'>


In [24]:
# 토크나이저 객체의 normalizer 속성에는 normalize_str() 메서드가 있다.
# 이 메서드는 정규화가 수행되는 방식을 확인하는데 사용할 수 있다.
# 이 예에서는 bert-base-uncased 체크포인트를 선택했기 때문에 정규화 과정에서 소문자화(lowercasing)를 수행하고 악센트를 제거한다.

print(tokenizer.backend_tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


#### - pre-tokenization

- 사전토큰화



In [25]:
# BERT 토크나이저를 사용하고 있기 때문에
# 사전 토큰화(pre-tokenization)에는 공백(whitespace)과 구두점(puntuation) 분할이 포함된다.

tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")
#--> 위치를 알려준다. 공백이 무시된다.

[('Hello', (0, 5)),
 (',', (5, 6)),
 ('how', (7, 10)),
 ('are', (11, 14)),
 ('you', (16, 19)),
 ('?', (19, 20))]

In [26]:
# GPT-2 토크나이저를 사용하는 경우:
# 공백과 구두점에서도 분할되지만 공백을 없애지 않고 Ġ 기호로 대체하므로 토큰을 디코딩하면 원래 공백을 복구할 수 있다.
# 또한 BERT 토크나이저와 달리 이 토크나이저는 이중 공백을 무시하지 않는다.

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")

[('Hello', (0, 5)),
 (',', (5, 6)),
 ('Ġhow', (6, 10)),
 ('Ġare', (10, 14)),
 ('Ġ', (14, 15)),
 ('Ġyou', (15, 19)),
 ('?', (19, 20))]